<img align="center" width="12%" style="padding-right:10px;" src="Images/Ccom.png">

# Tools for Ocean Mapping <a href="https://teams.microsoft.com/l/channel/19%3anLW3JzlSiEQLpIBtk3QaOqLYRlRECyjhkUkC3WH1qKM1%40thread.tacv2/General?groupId=e05e4f0f-c653-4060-854e-6a776126d431&tenantId=d6241893-512d-46dc-8d2b-be47e25f5666"><img src="Images/help.png"  title="Ask questions on Teams" align="right" width="10%" alt="Teams.com"></a><br><br> Lab 4: Handling Positioning Data 



***

## Creation of a Positioning Class

For this lab you will create a positioning class - you will use this positioning class to hold data contained in a HYPACK data file. 

The first step consists of creating some code that opens a HYPACK data file and extracts [NMEA-0183](https://www.nmea.org/content/STANDARDS/NMEA_0183_Standard) position strings from this file.

The next step is then to use a positioning class object to parse the data and store the data. 

The final step is to then create a write method to write the data to file

***

## Overview


### The Data

<img align="left" width="100%" style="padding-right:10px; padding-top:10px;" src="Images/rvcs_group.jpg"><br>

The data provided here were collected by the <ins>RV</ins> <ins>C</ins>oastal <ins>S</ins>urveyor. [RVCS](http://ccom.unh.edu/facilities/research-vessels/rv-coastal-surveyor) was a 12 m long research vessel operated by [CCOM/JHC](http://ccom.unh.edu) for hydrographic reasearch and education. RVCS has since been succeeded by the [RV Gulf Surveyor](http://ccom.unh.edu/facilities/research-vessels/rv-gulf-surveyor).

RVCS used a POS/MV 320 for positioning and orientation. In the case of this lab the positioning is RTK GPS data generated at 1Hz and representing the origin of the ship reference frame.

The RVCS was operating in the coastal waters of Maine in the immediate [vicinity of Portsmouth, New Hampshire](https://www.google.com/maps/@43.0714414,-70.3997656,9.67z). The data was obtained while RVCS was on station to obtain [benthic](https://oceanservice.noaa.gov/facts/benthic.html) video to support habitat mapping

As we has seen in class positioning is done relative to a model of the Earth. For horizontal positioning a biaxial ellipsoidal model such as associated to [WGS84](https://earth-info.nga.mil/index.php?dir=wgs84&action=wgs84) is used. The semi-major axis then describes the radius of the equator and the semi-minor axis defines the distance from the equatorial plane to the poles. You will learn more about this subject in the Geodesy course.

Normally, one of three vertical referencing approaches is used:

1. **Ellipsoid** Referenced – ERS,
2. **[Chart Datum](https://tidesandcurrents.noaa.gov/datum_options.html)** Referenced - CRS
3. **Geoid** Referenced - GRS

At this point in time not need elevation data, but we should ensure that we handle the data correctly so that in the future you use the same class for different purposes. Having the ability to use a class in more than software project is one of the key reasons that we use <ins>O</ins>bject <ins>O</ins>riented <ins>P</ins>rogramming


### Using CRS

Historically it was much more difficult to establish height above the ellipsoid (vertical position by **elevation**) than to establish the position along its surface (horizontal position by **geodetic latitude and longitude**). Thus often vessels were positioned by latitude and longitude using a positioning system. The vertical location was then estimated by combining water level observations at a tide gauge with an estimation of the draft of the vessel using some model for transferring the observed water level to your location.

A problem that occurs is that the sea state at the gauge and the vessel are different. This is addressed by filtering out the wave action at the gauge and observing  the vertical motions of the vessel due to sea state (**heave**). These heave measurements then need to be applied to the sensor observations.

Another problem that occurs is that the entire vessel will move up and down in the water as a function of the speed with which it moves through the water, a phenomenon known as **settlement**. Also its bow will either pitch up or down, a phenomenon known as **squat**. To correct the sensor observations it is then necessary to create a model of the settlement and squat as a function of speed through the water.

To correctly position the sonar transducers it was thus necessary to
1. Observe water levels at a tide gauge and transfer the observations to your location using some model
2. Observe speed through water using a **log** (device that measures speed though water) and estimate the resultant vertical motion through some settlement and squat model
3. Observe the heave and apply it to the observed data
4. Correct the observed data for the vertical motion induced through roll, pitch and yaw

### Using ERS

With the advent of positioning using **Global Navigation Satellite Sytems (GNSS)** such as GPS (USA), GLONASS (Russia), Beidou (China), Galileo (Eurpoean Union), IRNSS (India) and QZSS (Japan) it has become possible to directly estimate height above the ellipsoid with sufficient accuracy. This has obviated the need for observing many of the parameters other than the motion of the vessel, thereby greatly simplifying and enhancing the quality of positioning at sea. However, we still do need to correct for the motion of the vessel.

Note that if the depth data are to be referenced to a chart datum a model needs to be available to map the ellipsoid referenced data to chart datum. If such a model is not available water level data needs to be collected to enable the creation of one. However, this does not have to occur at the same time as depth data collection.


### Using GRS

In our case we will reference the data to the EGM2008 **geoid** which is associated to WGS84. Much like a reference ellipsoid a geoid is a model of the Earth, however, a far more complex one. A geoid is a model of an equipotential gravity surface whose potential is chosen so that it most closely fits Mean Sea Level (**MSL**). A fundamental property of a geoid is then that it is aligned everywhere to the local horizon plane (You will learn more about geoids in the Geodesy course).

Reference ellipsoids are usually defined so that they most closely resemble a geoid model. Differences between MSL and geoids typically range on the order of a few meters, whereas differences between ellipsoids and their associated geoids may range up to hundreds of meters.

The difference in height between a geoid and ellipsoid are known as undulations. In the case of the EGM2008 geoid there are well established undulations between it and the **WGS84** ellipsoid used by the Navstar Global Positioning System (**GPS**)

For simplicity, this lab does not entail the datum shift aspects required for CRS solution, merely reduction of the array-relative ranges to the geoid using the orthometric heights of the ship's reference frame origin, as provided by the POSMV 320


***

<img align="left" width="6%" style="padding-right:10px;" src="./Images/info.png"> 
## Lab Implementation in Python

This lab assumes a basic familiarity with Python at the level reached upon completion of the ePOM [**Programming Basics with Python for Ocean Mapping**](../python_basics) and [**Foundations of Ocean Data Science**](../ocean_data_science) sets of notebooks.

If you are already well versed in programming with Python you are free to move through the steps ahead of when they are assigned.

As part of the labs in this course you will be developing Python classes that may be used in other courses as well.

The goal is not too develop the most elegant and/or efficient code possible, but to have you write code that helps you achieve the learning objectives of the labs.

___

<img align="left" width="6%" style="padding-right:10px;" src="./Images/key.png"> 
Note that you are free to design your own code in Python or another language. If you decide to do so you have to demonstrate ***equivalency*** of the deliverables. Also, we will be happy to support any coding questions for those following the (Python) instructions provided here, but can not provide the same level of service for those using different algorithms and/or programming languages.

***


# Step 0: Creating Classes

In this step you will a class that can hold positioning data and associated metadata. 

After completion of this step an `object` of this class type will not do much yet. At this point you will only create the class definitions and their data attributes. 

***

In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
import numpy as np

sys.path.append(os.getcwd())  # add the current folder to the list of paths where Python looks for modules 
abs_path=os.path.abspath(os.path.curdir)

Similarly to what is described in the [*Class Definition*](../python_basics/008_A_Class_as_a_Data_Container.ipynb#Class-Definition) notebook, we start by creating a class definition. In this case, the class will be named `Position` and will have a `"""A Class for handling Position Data"""` docstring.

***

## 0.0 Creating the Class `Position`

<img align="left" width="6%" style="padding-right:10px;" src="./Images/key.png">

In this step you will create a class `Position`.

<img align="left" width="6%" style="padding-right:10px;" src="./Images/test.png">

Modify the empty `position.py` file located in the `mycode` folder to successfully execute the code in the code cell below.

To define the class you may use:
    
    class Position:
        """A Class for handling Position Data"""

        def __init__(self):
            pass


The print statement should result in the string: `The object type is <class 'mycode.position.Position'>` You may achieve this by adding the \_\_str\_\_ method

    def __str__(self): 

In [3]:
from mycode.position import Position
from datetime import datetime

# positions
pos = Position()
print("The object type is %s" % (type(pos)))

The object type is <class 'mycode.position.Position'>


***

## 2.1 Class Initialization and Attributes

We can now add a few attributes to the previously defined `Position` class:

* Four lists (named `times`, `latitudes`, `longitudes` and `heights` respectively
* A string called `data_path`
* a numpy array called proj_pos
* A `metadata` dictionary with the following pairs of key and value:
  * `"geodetic_units"`: `"rad"` for the unit the geodetic latitudes and longitudes
  * `"height_units"`: `"m"` for the units of elevations
  * `"proj_units"`: `"m"` for the units of projected coordinates
  * `"geoid_name"`: `None` For the name of the geoid used
  * `"ellipsoid_name"`: `None` For the name of the ellipsoid used
  * `"height_relative_to"`: `None` For the name of the vertical datum used
  * `"time_basis"`: `"UTC"` The times are `Universal Time Coordinated`
  * `"proj_str"`: '"Unknown"' For the string used for projecting coordinates
  
An empty list may be declared by a statement such as:

    my_list = []

Similarly an empty string is declared by:

    my_string = str()
    
As you may see the `numpy` package is imported as `np`. Thus to create an empty `numpy` array we may use:

    my_np_array = np.array([])
    
Finally an empty dictionary is declared by:
    
    my_dict = dict()


As described in [*Class Initialization and Attributes*](../../python_basics/008_A_Class_as_a_Data_Container.ipynb#Class-Initialization-and-Attributes), the 0initialization of the class attributes happens in the *magic* `__init__(self)` method. 

## 2.2 The String Representation Method

Create a `__str__` method for the Position class in similar fashion as the one you created for the Water Level class. We also want to print the start and end time and the geographic area covered. However, just like for the Water Level class, at this point there are no data yet. To get around this you must make the printing of the time and positioning data conditional upon data being present. 

print the `pos` object 

In [4]:
print(pos)

geodetic_units: rad
height_units: m
proj_units: m
geoid_name: None
ellipsoid_name: None
height_relative_to: None
time_basis: UTC
proj_str: None



The output should read:

    geodetic_units: rad
    height_units: m
    proj_units: m
    geoid_name: None
    ellipsoid_name: None
    height_relative_to: None
    time_basis: UTC
    proj_str: None

# 3 Geodetic Referencing

Before we go on we need to discuss positioning a little.

<img align="center" width="50%" style="padding-right:10px;" src="Images/GeoidEllipsoid.png">

In this lab we will create a  `ParseNMEA0183_GGA()` method which we will use to read positioning data. However, positioning data is useless without knowing what the positioning is relative to. Clearly we want to position ourselves relative to the Earth, which means that we need some geometrical description of it. We should not use the topography of the Earth as the model, otherwise both *Mount Everest* and the *Marianas Trench* would have an elevation of 0 m. Commonly people refer to elevations relative to sea level - however this is problematic as not the entire Earth is covered by an ocean. In reality a proxy is used to Mean Sea Level (**MSL**), namely an equipotential gravity field that is a best fit to MSL. This equipotential field is known as the **Geoid** which in the figure above is represented in blue. 

For vertical positioning the geoid is a good datum as the distance from it predicts the potential energy of an object of given mass through $E_{pot}=mgh$, where $h$ is the orthogonal distance from the surface of the geoid. 
The height data in your data file are relative to the **EGM08** geoid model. 

For calculating distances along the surface of the geoid model ambiguities may occur due to its irregular nature, thus we usually use a simple geometric shape that approximates the geoid and on which we can define distances and horizontal coordinates uniquely, such a model is shown in orange in the figure above. 

<img align="center" width="50%" style="padding-right:10px;" src="Images/Ellipsoid.png">

In this case the data is relative to a **geodetic datum**, consisting of an oblate ellipse of revolution - that is: a 2D ellipse that is rotated around its semi-minor axis, as shown in the figure above. The origin of these models sit on the modeled center of gravity of the Earth, the semi-minor axis is aligned to the Earth's rotational axis and the semi-major axis sweeps the equatorial plane. 

There are many possible geodetic datums that may be used. Historically we used ellipsoid that locally best fit the geoid, but with the advent of Global Navigation Satellite Systems (**GNSS**) a shift towards the use of global datums has taken place. The **GPS** system uses the **WGS84** geodetic datum, which is what coordinates in your data file are relative to.

<img align="center" width="80%" style="padding-right:10px;" src="Images/Geodetic_LL.png">

The coordinates given to you are knowns as **geodetic latitudes and longitudes**, that is: latitudes and longitudes relative to a geodetic datum. First we will have to define what meridians and parallels are before we can express what latitudes and longitudes are.

The **poles** of the geoid are where the semi-minor axis intersects the ellipsoid. A Meridian is then the intersection of the plane containing both the poles and a point of interest. This plane is inherently orthogonal to the equatorial plane and the intersection with the ellipsoid is an ellipse referred to as a meridian. All points in this plane on one side of the semi-minor axis are said to have the same longitude, all points on the other side are offset by $180^{\circ}$. The difference in **geodetic longitude** between two points is then the angle in the equatorial plane between the meridians associated to them. To provide a useful datum a zero meridian has been chosen, it goes through a point near the **Greenwich** Royal observatory in England. The figure above left then illustrates longitude $\lambda$ relative to the Greenwich meridian $\lambda_0$. 

All points in an equidistant plane from the equatorial plane are said to have the same latitude. There are many 'flavors' of latitude, but the one that is most commonly used for positioning is named **geodetic latitude**. Geodetic latitude is defined by the line tangent to the meridian associated to a point of interest. Geodetic latitude $\varphi$ is then the angle between the line orthogonal to the tangent and the equatorial plane $\varphi_0$, as shown in the figure above on the right

## 3.0 Set the reference ellipsoid and geoid

In the HYPACK text file containing the coordinates no meta-data indicating the georeferencing parameters is included. In this case we are using orthometric heights relative to EGM08, and geodetic latitudes and longitudes relative to WGS84. <br>

<img align="left" width="6%" style="padding-right:10px;" src="Images/test.png"><br>
Create a `ParseNMEA0183_GGA()` method for the `Position` class to parse GGA records. The method will take and set the metadata parameters for `geoid_name`, `ellipsoid_name`, and `height_relative_to` (as the heights are orthometric this should be set to "geoid")
<br><br>
In order to do this you may copy the template below into the python.py file.
    
                
    def ParseNMEA0183_GGA(self, dt_str, geoid_name, ellipsoid_name, height_relative_to, date)

        # Get the GGA string and tokenize it
        gga_data = dt_str.split(',')

        # verify that we have a GGA string
        if not gga_data[0][-3:] == "GGA":
            raise RuntimeError(
                    'ParseNMEA0183_GGA: argument `dt_str` must be a GGA message')

        self.metadata["geoid_name"] = geoid_name
        self.metadata[...] = ellipsoid_name
        self.metadata[...] = height_relative_to

You do not have to do anything other than filling in the code where the ellipsis (...) are. Once you have done this the output of the code cell below should match 


In [14]:
pos.ParseNMEA0183_GGA('GPGGA',"EGM08","WGS84", "geoid")
print("pos:\n", pos)

GPGGA
pos:
 geodetic_units: rad
height_units: m
proj_units: m
geoid_name: EGM08
ellipsoid_name: WGS84
height_relative_to: geoid
time_basis: UTC
proj_str: None



# 4 Parsing HYPACK data

In our case the NMEA 0183 GGA records holding the position data are embedded in the HYPACK file - we will need to extract them and then parse them with the `ParseNMEA0183_GGA` method we just created. For this we will create the `read_hypack_raw_file` method that does the file handling for the HYPACK files, finds the GGA strings and then calls the `ParseNMEA0183_GGA` method. Off course we will need the `ParseNMEA0183_GGA` to actually make it parse the data

    def read_hypack_raw_file(self, fullpath):
        
        # This function will currently only function provided that there are GGA sentences 
        # in the records.
        # You may update the function to include other positioning messages as well, but 
        # this is outside the scope of the class
                        
        # Check the File's existence
        if os.path.exists(fullpath):
            self.data_path = fullpath
            print('Opening GNSS data file:' + fullpath)
        else:  # Raise a meaningful error
            raise RuntimeError('Unable to locate the input file' + fullpath)
            
        # Open, read and close the file
        hypack_file = open(fullpath)
        hypack_content = hypack_file.read()
        hypack_file.close    

        # Split the file in lines
        hypack_records = hypack_content.splitlines()
        
        # Go through the header lines to find the date of the survey 
        # (not contained in the GGA records)
        
        lines_parsed=0
        for hypack_record in hypack_records:
            
            # Check for the time and date
            
            if hypack_record[:3].lower() == "tnd":
                hypack_datetime=datetime.strptime(hypack_record[4:23], "%H:%M:%S %m/%d/%Y")
                
                print("HYPACK RAW Header start time and date: " + hypack_datetime.ctime())
                
            # Keep track of the lines parsed
            lines_parsed+=1

            # Stop going through the records if the record starts with the string eoh 
            # (End Of Header)
            if hypack_record[:3].lower() == "eoh":
                break         
        
        # Keep track of the number of GGA records found
        
        num_gga_recs=0
        
        for hypack_record in hypack_records[lines_parsed:]:

            if hypack_record[19:22] == "GGA":
                gga_data=hypack_record.split()[3]
                print(gga_data)
                
___
## 4.0 Parsing HYPACK data

Copy the code above and add it to your `Positions` class. Make sure to inspect the code and see whether you understand what it does. Test its functionality by executing the code cell below.

In [15]:
# The Positions class is designed to be able to hold data from multiple files
# this may create memory management issues while developing. 
# To avoid this we will simply create a new Position object every time this 
# cell gets executed

positions = Position()
positions.read_hypack_raw_file(abs_path+'/Data/000_1029.RAW')

Opening GNSS data file:/home/jupyter-semmed/ESCI_872/Data/000_1029.RAW
HYPACK RAW Header start time and date: Tue Jul  3 10:29:07 2012
$GPGGA,143026,4303.0241,N,07038.7748,W,1,09,0.9,10.2,M,-32.4,M,,*4A
$GPGGA,143027,4303.0241,N,07038.7748,W,1,09,0.9,10.3,M,-32.4,M,,*4A
$GPGGA,143028,4303.0241,N,07038.7749,W,1,09,0.9,10.4,M,-32.4,M,,*43
$GPGGA,143029,4303.0241,N,07038.7748,W,1,09,0.9,10.3,M,-32.4,M,,*44
$GPGGA,143030,4303.0241,N,07038.7746,W,1,09,0.9,10.3,M,-32.4,M,,*42
$GPGGA,143031,4303.0241,N,07038.7747,W,1,09,0.9,10.2,M,-32.4,M,,*43
$GPGGA,143032,4303.0241,N,07038.7746,W,1,09,0.9,10.3,M,-32.4,M,,*40
$GPGGA,143033,4303.0241,N,07038.7743,W,1,09,0.9,10.3,M,-32.4,M,,*44
$GPGGA,143034,4303.0241,N,07038.7743,W,1,09,1.0,10.4,M,-32.4,M,,*4C
$GPGGA,143035,4303.0241,N,07038.7744,W,1,09,0.9,10.6,M,-32.4,M,,*40
$GPGGA,143036,4303.0241,N,07038.7743,W,1,09,0.9,10.2,M,-32.4,M,,*40
$GPGGA,143037,4303.0239,N,07038.7741,W,1,09,0.9,10.0,M,-32.4,M,,*4E
$GPGGA,143038,4303.0240,N,07038.7742,W,1,09,0.9,1

    
    Opening GNSS data file:/home/jupyter-semmed/ESCI_872/Data/000_1029.RAW
    HYPACK RAW Header start time and date: Tue Jul  3 10:29:07 2012
    $GPGGA,143026,4303.0241,N,07038.7748,W,1,09,0.9,10.2,M,-32.4,M,,*4A
    $GPGGA,143027,4303.0241,N,07038.7748,W,1,09,0.9,10.3,M,-32.4,M,,*4A
    $GPGGA,143028,4303.0241,N,07038.7749,W,1,09,0.9,10.4,M,-32.4,M,,*4
    ...

___
## 4.1 Calling the GGA String Parser

Success! Well, partially. The bit off code given to you strips out the GGA strings from the HYPACK file, but we're not doing anything yet with them. It also should be pointed out at this time that GGA strings have a severe limitation, they do not include a date! We will deal with this in other courses, but its worth while pointing out here.

GGA strings are so commonly used that they deserve their own method to be parsed. That way we do not have to recreate a GGA parser for any type of data file that has GGA strings embedded in it. At this point it may be worthwhile considering whether we should create a class for handling NMEA-0183 data. In our case the handling of the data in GGA strings is so specific to positioning that it is worthwhile to handle the parsing as part of the Position class.

Replace the print(gga_data) in your read_hypack_raw_file() with a call to the ParseNMEA0183_GGA() method. Note that when you call a method from within the object itself you should use:

    self.name_of_the_method_I_am_calling() 

and off-course replace name_of_the_method_I_am_calling with whatever the name of the method that you're calling is, and supply the appropriate arguments including the date.

In [42]:
# The Positions class is designed to be able to hold data from multiple files
# this may create memory management issues while developing. 
# To avoid this we will simply create a new Position object every time this 
# cell gets executed

positions = Position()
positions.read_hypack_raw_file(abs_path+'/Data/000_1029.RAW')

Opening GNSS data file:/home/jupyter-semmed/ESCI_872/Data/000_1029.RAW
HYPACK RAW Header start time and date: Tue Jul  3 10:29:07 2012
2012-07-03 14:30:26
2012-07-03 14:30:27
2012-07-03 14:30:28
2012-07-03 14:30:29
2012-07-03 14:30:30
2012-07-03 14:30:31
2012-07-03 14:30:32
2012-07-03 14:30:33
2012-07-03 14:30:34
2012-07-03 14:30:35
2012-07-03 14:30:36
2012-07-03 14:30:37
2012-07-03 14:30:38
2012-07-03 14:30:39
2012-07-03 14:30:40
2012-07-03 14:30:41
2012-07-03 14:30:42
2012-07-03 14:30:43
2012-07-03 14:30:44
2012-07-03 14:30:45
2012-07-03 14:30:46
2012-07-03 14:30:47
2012-07-03 14:30:48
2012-07-03 14:30:49
2012-07-03 14:30:50
2012-07-03 14:30:51
2012-07-03 14:30:52
2012-07-03 14:30:53
2012-07-03 14:30:54
2012-07-03 14:30:55
2012-07-03 14:30:56
2012-07-03 14:30:57
2012-07-03 14:30:58
2012-07-03 14:30:59
2012-07-03 14:31:00
2012-07-03 14:31:01
2012-07-03 14:31:02
2012-07-03 14:31:03
2012-07-03 14:31:04
2012-07-03 14:31:05
2012-07-03 14:31:06
2012-07-03 14:31:07
2012-07-03 14:31:08
2012-

Note that the output of the exact same code as the previous code cell has changed (and if you re-execute that code cell the output there will change too):

    Opening GNSS data file:/home/jupyter-semmed/ESCI_872/Data/000_1029.RAW
    HYPACK RAW Header start time and date: Tue Jul  3 10:29:07 2012


___
## 4.1 Parsing the Time

We pointed out at this time that GGA strings have a severe limitation, they do not include a date. For now we will just live with it and use a `timedelta` and supply a date - in this case the date of the header of the HYPACK file, which is contained in the local variable `hypack_datetime`. When you realize that a file may be opened before midnight and closed after you will understand that this is problematic. Since many survey ships operate 24/7 this is not a problem we should ignore, but we will to keep the scope of the lab limited.

Add the following code to the `ParseNMEA0183_GGA()` method:

        # Determine the time of day from both the header and the GGA string

        gga_timedelta=timedelta(hours=int(gga_data[1][0:2]), \
                                 minutes = int(gga_data[1][2:4]), \
                                 seconds = int(gga_data[1][4:6]))
        
        # Set the time of the date to midnight
        
        date = datetime(date.year, date.month, date.day, 0, 0, 0)

        self.times.append(date + gga_timedelta)


In [44]:
positions = Position()
positions.read_hypack_raw_file(abs_path+'/Data/000_1029.RAW')
print(positions.times[0])

Opening GNSS data file:/home/jupyter-semmed/ESCI_872/Data/000_1029.RAW
HYPACK RAW Header start time and date: Tue Jul  3 10:29:07 2012
2012-07-03 14:30:26


    
    Opening GNSS data file:/home/jupyter-semmed/ESCI_872/Data/000_1029.RAW
    HYPACK RAW Header start time and date: Tue Jul  3 10:29:07 2012
    2012-07-03 14:30:26


___
## 4.2 Parsing the Latitude

In similar fashion to time we should add the latitude, Note that the latitude hemisphere is given in the 4th field and that we want the range to be $[-\pi/2,\pi/2]$

Add the following code to the `ParseNMEA0183_GGA()` method:


        # Parse the latitude
        if gga_data[3].lower() == "n":
            lat = float(gga_data[2][0:2])+float(gga_data[2][2:])/60.
        else:
            lat = -float(gga_data[2][0:2])-float(gga_data[2][2:])/60.
        self.latitudes.append(lat)

___
## 4.3 Parsing the Longitude

In similar fashion to latitude we should add the longitude, in this case we want the range to be $[-\pi,\pi]$

Add the following code to the `ParseNMEA0183_GGA()` method:


        # Parse the longitude
        if gga_data[5].lower == ...:
            lon = ...
        else:
            lon = ...
        self.longitudes.append(lon)
            
and update it by filling in the ellipsis

In [46]:
positions = Position()
positions.read_hypack_raw_file(abs_path+'/Data/000_1029.RAW')
print(positions.longitudes[0])

Opening GNSS data file:/home/jupyter-semmed/ESCI_872/Data/000_1029.RAW
HYPACK RAW Header start time and date: Tue Jul  3 10:29:07 2012
-70.64624666666667


___
## 4.4 Parsing the Quality

As you have learned from researching the NMEA messages there is a quality indicator
add an list `qualities` to the `Positions` class, and add the following code to the `ParseNMEA0183_GGA()` method:

        # Parse the GNSS Quality indicator
        q = ...
        self.qualities.append(...)
            
and update it by filling in the ellipsis

In [49]:
positions = Position()
positions.read_hypack_raw_file(abs_path+'/Data/000_1029.RAW')
print(positions.qualities[0])

Opening GNSS data file:/home/jupyter-semmed/ESCI_872/Data/000_1029.RAW
HYPACK RAW Header start time and date: Tue Jul  3 10:29:07 2012
1


___
## 4.5 Parsing the Number of Satellites

Next create a num_sats list and add the number of GNSS satellites in view

        # Parse the number of GNSS satellites used for the solution
        n_sats = ...
        self.num_sats.append(...)
            
and update it by filling in the ellipsis

In [49]:
positions = Position()
positions.read_hypack_raw_file(abs_path+'/Data/000_1029.RAW')
print(positions.num_sats[0])

Opening GNSS data file:/home/jupyter-semmed/ESCI_872/Data/000_1029.RAW
HYPACK RAW Header start time and date: Tue Jul  3 10:29:07 2012
1


___
## 4.6 Parsing the Horizontal Dilution of Precision

Next create a hdops list and add the number of GNSS satellites in view

        # Parse the HDOP Quality indicator
        hdop = ...
        self.hdops.append(...)
            
and update it by filling in the ellipsis

In [49]:
positions = Position()
positions.read_hypack_raw_file(abs_path+'/Data/000_1029.RAW')
print(positions.qualities[0])

Opening GNSS data file:/home/jupyter-semmed/ESCI_872/Data/000_1029.RAW
HYPACK RAW Header start time and date: Tue Jul  3 10:29:07 2012
1


___
## 4.7 Parsing the Orthometric Height

Add the orthometric height to the heights list

        # Parse the orthometric height 
        height = float(...)
        self.hdops.append(...)
            
and update it by filling in the ellipsis

# 5 Create a Hotlink File

The work is already done for you! Just add the following code to the `Position` class:
    
    def write_hotlink(self, hotlink_path):
        
        fullpath, _ = os.path.splitext(self.data_path)
        
        fullpath = fullpath + "_pos.txt"
       
        # Check the File's existence

        if os.path.exists(fullpath):
            # Let the user now we are overwriting an existing file
            self.data_path = fullpath
            print('Overwriting file: ' + fullpath)
        else:  # Let the user know we are writing to a file
            print('Writing to file: ' + fullpath)
            
        output_file = open(fullpath, mode="w")  # mode="w" to open the file in writing mode
        
        # Write the header

        output_file.write('date time latitude longitude path\n')
        
        # Determine the duration associated to the positions in this object
        
        start_time = min(self.times)
        duration = max(self.times) - start_time
    
        # For each position write the date time longitude latitude path and fraction
        
        for i in range(0,len(self.times)):
            fraction = (self.times[i] - start_time) / duration
            line_content=str(self.times[i]) \
            + " %012.8f %013.8f %s?%f\n" % \
            (self.latitudes[i], self.longitudes[i], hotlink_path, fraction)
            output_file.write(line_content)

***

# 6 Teams

In the General channel of 872 on Teams post a message discussing a single aspect of this assignment

<img align="left" width="6%" style="padding-right:10px; padding-top:10px;" src="Images/refs.png">

## Useful References

* [The official Python 3.6 documentation](https://docs.python.org/3.6/index.html)
  * [Classes](https://docs.python.org/3.6/tutorial/classes.html)
  * [String Representation Method](https://docs.python.org/3.6/reference/datamodel.html?highlight=repr#object.__str__)
* [Memory address](https://en.wikipedia.org/wiki/Memory_address)
* [ePOM: Programming Basics with Python](https://github.com/hydroffice/python_basics)
* [ePOM: Foundations of Ocean Data Science](https://github.com/hydroffice/ocean_data_science)
* [NMEA-0183](https://www.nmea.org/content/STANDARDS/NMEA_0183_Standard)
* [WGS84](https://earth-info.nga.mil/index.php?dir=wgs84&action=wgs84)
* [Chart Datum](https://tidesandcurrents.noaa.gov/datum_options.html)


<img align="left" width="5%" style="padding-right:10px;" src="Images/email.png">

*For issues or suggestions related to this notebook that should not be addressed on Teams, write to: semmed@ccom.unh.edu*

Python code and Notebook implementation by Semme J. Dijkstra